In [1]:
import os
import sys

import numpy as np
from math import sqrt
from collections import defaultdict

sys.path.insert(0,os.path.abspath('../'))
from FP_Utilities import Read_E_Bins, Build_Nagy_Weighted_FPs, Build_Spline_Weighted_FPs, Build_GEF_Weighted_FPs
from FP_Utilities import Read_Fission_Spectrum, writeout

rootdir=os.path.abspath(os.getcwd())

In [2]:
# Determine fission fraction by isotope
# From ETA Ouput - Not explicitly required for program
tot=1.32237E-05
tot_234=1.02627E-07
tot_235=1.29185E-05
tot_236=1.69356E-08
tot_238=1.78433E-07
wf_234=0.0093898
wf_235=0.93217
wf_236=0.003
wf_238=0.05544
print "235",tot_235/tot
print "238",(tot_234+tot_236+tot_238)/tot   # 234 and 236 lumped with 238 because minor and threshold fissioners


235 0.976920226563
238 0.0225349637393


In [3]:
# User Inputs
#---------------------------------------------------------------------------------------#  
fiss235Name = 'Data/E_fiss_235.csv'   # Fission spectra input file
fiss238Name = 'Data/E_fiss_238.csv'   # Fission spectra input file
binsName = 'Data/Bins.csv'   # Bin structure input file
fp235Name = '../235_data.xlsx'  # 235 energy dependent data - must contain same isotopes as 238
fp238Name = '../238_data.xlsx'  # 238 energy dependent data - must contain same isotopes as 235
gef235Dir = '../../Validation/235Watt' # Path to GEF data
gef238Dir = '../../Validation/238Watt' # Path to GEF data

f235 = 0.976920226563       # fraction of fissions caused by 235 - f_235+f_238 should equal 1
f238 = 0.0225349637393  # fraction of fissions caused by 238 - f_235+f_238 should equal 1
#---------------------------------------------------------------------------------------#  

In [4]:
### Nagy Based Predictions ###

# Read in the energy bin structure
(lower_bins,upper_bins,bins)=Read_E_Bins(binsName)

# Calculate 235 data
(fiss_235_e,fiss_235)=Read_Fission_Spectrum(fiss235Name)
y_235, err_235=Build_Nagy_Weighted_FPs(fp235Name,bins,f235,fiss_235)

# Calculate 235 data
(fiss_238_e,fiss_238)=Read_Fission_Spectrum(fiss238Name)
y_238, err_238=Build_Nagy_Weighted_FPs(fp238Name,bins,f238,fiss_238)

# Combine the data into a single data set
y = {}
err = {}
absErr = {}
out = []
for A in y_235.keys():
    y[A]=y_235[A]+y_238[A]
    err[A]=sqrt(err_235[A]**2+err_238[A]**2)
    absErr[A]=err[A]*y[A]
    out.append((A,y[A],absErr[A]))

# Output Results
print y, absErr
writeout("ETA_Nagy_fy.csv",sorted(out,key=lambda l:l[0]))

{97: 5.7321644790991284, 99: 5.6457635034788316, 133: 6.3834161362530066, 161: 0.0010790371615357198, 140: 5.6842336722614686, 141: 5.4727031139368734, 111: 0.26405449483673388, 115: 0.26431775916676636, 113: 0.18790081000415867, 147: 2.1117247468846134, 151: 0.47250296097435707, 153: 0.17663559923480682, 156: 0.026941196956127279, 95: 6.1517512054802417} {97: 0.091867072660467911, 99: 0.090159382676315819, 133: 0.13435263172529863, 161: 4.5479257271362311e-05, 140: 0.069603562886220244, 141: 0.099337442469776813, 111: 0.0069196031114892092, 115: 0.011721290449753652, 113: 0.01151184198710151, 147: 0.032774564512270669, 151: 0.016908489687956312, 153: 0.0077704402344916735, 156: 0.00093374597829739181, 95: 0.091349610716997515}


In [5]:
### Spline Based Predictions ###

# Read in the energy bin structure
(lower_bins,upper_bins,bins)=Read_E_Bins(binsName)

# Calculate 235 data
(fiss_235_e,fiss_235)=Read_Fission_Spectrum(fiss235Name)
y_235=Build_Spline_Weighted_FPs(fp235Name,bins,f235,fiss_235)

# Calculate 235 data
(fiss_238_e,fiss_238)=Read_Fission_Spectrum(fiss238Name)
y_238=Build_Spline_Weighted_FPs(fp238Name,bins,f238,fiss_238)

# Combine the data into a single data set
y=defaultdict(list)
out=[]
for A in y_235.keys():
    y[A]=np.sum(np.asarray(y_235[A])+np.asarray(y_238[A]))
    out.append((A,y[A],0.0))
print y

# Output Results
writeout("ETA_Spline_fy.csv",sorted(out,key=lambda l:l[0]))

defaultdict(<type 'list'>, {97: 5.6991902456724617, 99: 5.5294806467283788, 133: 6.3636045019618726, 161: 0.0012593323879698272, 140: 5.6589784650358261, 141: 5.7190188157666473, 111: 0.26452237170147558, 115: 0.25462022906797366, 113: 0.2065683115379926, 147: 2.151837841132616, 151: 0.43180661108312535, 153: 0.1769282676051559, 156: 0.029618639000638557, 95: 6.1960554573677165})


In [5]:
### GEF Based Predictions ###

# Read in the energy bin structure
(lower_bins,upper_bins,bins)=Read_E_Bins(binsName)

# Calculate 235 data
y_235={}
err_235={}
absErr={}
(y_235,err_235)=Build_GEF_Weighted_FPs(bins,f235*fiss_235,gef235Dir)

# Calculate 238 data
y_238={}
err_238={}
(y_238,err_238)=Build_GEF_Weighted_FPs(bins,f238*fiss_238,gef238Dir)

# Combine data into a single data set
y={}
err={}
out=[]
for A in y_235.keys():
    try:
        y[A]=y_235[A]+y_238[A]
        err[A]=sqrt(err_235[A]**2+err_238[A]**2)
        absErr[A]=y[A]*err[A]
        out.append((A,y[A],err[A],absErr[A]))
    except KeyError as e:
        print "Key error: {0} not found for both isotopes.".format(A)

# Output Results
writeout("ETA_GEF_fy.csv",sorted(out,key=lambda l:l[0]))





47 46 46
Key error: 58 not found for both isotopes.
Key error: 54 not found for both isotopes.
Key error: 52 not found for both isotopes.
Key error: 180 not found for both isotopes.
Key error: 181 not found for both isotopes.
